### Character-Level RNNs

Diese Notebooks wurden inspiriert von dem großartigen Blogartikel "The Unreasonable Effectiveness of Recurrent Neural Networks" (http://karpathy.github.io/2015/05/21/rnn-effectiveness/).

In diesem Notebook wollen wir ihnen zeigen, wie man ein rekurrentes neuronales Netzwerk (RNN) auf Buchstabenebene benutzt, um Texte zu erzeugen, in dem man z.B. eine kurze Startsequenz vorgibt, und das trainierte RNN diese fortsetzen lässt. Dabei gibt der erzeugte Text die statistische Stuktur der Trainingstexte wieder.

Aber wie genau sieht so ein Character-Level-RNN aus?

In seiner fundamentalen Form so:

<img src="./charseq.png">

Das RNN besitzt ein Hidden-Layer (grün, in diesem Fall aus drei Neuronen bestehend). Der neue Wert des Hidden Layers wird - anhand gelernter Netzwerkgewichte - aus einem (als Zahlenfolge codierten) Buchstabeninput (rot) *und* dem vorhergehenden Wert des Hidden Layers berechnet. So erlauben es die Aktivierungen der drei Hidden Neurons in diesem Beispiel dem Netzwerk, sich an die vorhergehende Zeichenkette zu erinnern (falls es im Training eine entsprechende Repräsentation der zuvor präsentierten Daten lernen kann). Das Netzwerk erzeugt bei jeder Buchstabenpräsentation einen Output (blau). Da dieser vom Hidden-State des Netzerkes abhängt, besitzt die Output-Funktion nicht nur die Information über den momentanen Input-Buchstaben, sondern auch eine komprimierte Repräsentation der vorhergehenden Zeichenkette.

In dem Beispiel hier im Bild und in diesem Notebook besitzt das Output-Layer genauso viele Neurone, wie das Alphabet auf dem das Netzwerk trainiert wurde. Im Beispiel im Bild sind das nur vier Buchstaben ('h','a','l','o'). ("char" ist kurz für "character").

Das selbe gilt für das Input-Layer. Während das Output-Layer trainiert wird, ist in diesem Beispiel Aktivierung des Input-Layers fest vorgegeben: Es gibt jeweils ein Neuron, das den gerade präsentierten Input-Buchstaben repräsentiert. Dieses hat die Aktivierung 1.0, alle anderen haben die Aktivierung 0.0. Man kann dieses sogenannte "Input-Embedding" (d.h. wie man eine endliche Zahl einzelner Inputs in den Raum einbettet, der von den Aktivierungen des Input-Layers aufgespannt wird) auch lernen (unser Netzwerk weiter unten wird das tun).

Die Netzwerkgewichte bestimmen, wie aus dem Input und dem Hidden State der neue Hidden State berechnet wird, und wie aus dem Hidden State der Output berechnet wird. Die Zielfunktion, mit der diese Netzwerkgewichte trainiert wurden zielte hier darauf ab, dass das RNN die Wahrscheinlichkeitsverteilung über den jeweils nächsten Buchstaben in einer Zeichenkette lernt. Aber wie funktioniert das?

Die Aktivität jedes Output-Neurons wird als Score für einen Buchstaben im trainierten Alphabet interpretiert. Dieser Score beschreibt die Wahrscheinlichkeit, dass der entsprechende Buchstabe als nächstes in der Zeichenkette folgt (höher = größere Wahrscheinlichkeit). Betrachten wir das Beispiel im Bild. Dem Netzwerk wurden nacheinander die Buchstaben "h", "a", "l" und nochmal "l" präsentiert. Dabei wurde jeweils aus dem Input und dem alten Hidden State der neue Hidden State berechnet, und aus dem Hidden State die Aktivierung der vier (eines pro Buchstabe) Output-Neurone.

Die Zielfunktion, die im Training optimiert wurde, funktioniert nun so: Die Aktivierung eines Output-Neurons wird als Score dafür interpretiert, dass der entsprechende Buchstabe als nächstes in der Sequenz kommen wird.

Besitzen wir nun die komplette Sequenz, in diesem Beispiel "hallo", dann können wir den Output mit dieser tatsächlichen, sogenannten *target*, Sequenz vergleichen. In dem Bildbeispiel sind die Output-Scores, die den tatsächlichen nächsten Zeichen entsprechen *rot* eingefärbt, die Output-Scores für die andern, nicht-nachfolgenden Buchstaben *schwarz*.

Die Zielfunktion, die wir nun benutzen, übersetzt die Scores in eine Wahrscheinlichkeitsfunktion, und zwar so, dass höhere Scores höheren Wahrscheinlichkeiten entsprechen. Weiter wird darauf geachtet, dass alle Wahrscheinlichkeiten positiv sind und sich auf 1.0 aufsummieren. Dann wird die Wahrscheinlichkeit für den tatsächlich folgenden Buchstaben genommen und diese so in eine Kostenfunktion verpackt, dass niedrigere Werte dieser Kostenfunktion einer höheren Wahrscheinlichkeit für den tatsächlichen Buchstaben entsprechen. Der Mittelwert dieser Kostenfunktion für ein _Batch_ (d.h. für eine kleine Sammlung, z.B. 32, Sequenzen einer bestimmten Länge, z.B. 100 Zeichen, aus den Trainingsdaten) wird ausgerechnet und bildet die Zielfunktion (oft auch als Kostenfunktion oder "loss" bezeichnet). Diese wird nun über einen gradientenbasierten Optimierungsprozess minimiert.

**Anschaulich bedeutet das, dass während des Netzwerktrainings die Netzwerkgewichte so verändert werden, dass sich die Output-Scores für die tatsächlich folgenden Buchstaben in der Trainingssequenz erhöhen, und die für die nicht nachfolgenden Buchstaben verringern.**

Wenn das Training abgeschlossen ist, kann man dem Netzwerk nun einen kurze Startzeichenkette präsentieren. Der Output, den das Netzwerk dann für das letzte Zeichen in der Startzeichenkette erzeugt, kann dann in die Wahrscheinlichkeitsverteilung für den nächsten Buchstaben umgewandelt werden. Dann kann man mit dieser Wahrscheinlichkeitsverteilung "würfeln", d.h. zufällig, aber entsprechend der vom Netzwerk vorhergesagten Wahrscheinlichkeiten, einen nächsten Buchstaben ziehen. Diesen Buchstaben kann man dann wieder dem Netzwerk präsentieren. Der entsprechende Output kann dann wieder in die Wahrscheinlichkeitsverteilung für den nächsten Buchstaben umgerechnet werden, damit kann dann wieder ein neuer Buchstabe gezogen werden, und so weiter.

Bevor wir uns mit dem Training auf eigenen Daten beschäftigen, wollen wir uns erst einmal anschauen, wie man mit einem trainierten Netzwerk auf diese Weise Texte erzeugen bzw. fortschreiben lassen kann.

Zunächst importieren wir aber wieder PyTorch, die Bibliothek, die wir benutzen werden, um die mathematischen Operationen auf der Graphikkarte auszuführen, und um automatisch den Gradienten der Zielfunktion auszurechnen und damit die Netzwerkgewichte zu optimieren.

In [ ]:
# PyTorch (https://pytorch.org/docs/stable/)
# eine (relativ) einfach zu benutzende Bibliothek, um neuronale Netze
# mit Graphikkartenunterstützung zu erstellen, trainieren und anzuwenden
import torch
import torch.nn as nn
import torch.nn.functional as F
import pickle

# Eine kategorielle Wahrscheinlichkeitsverteilung (Verallgemeinerung eines Würfel 
# auf "unfaire" (d.h. beliebig aufgeteilte) Wahrscheinlichkeiten für beliebig viele "Seiten")
from torch.distributions import Categorical
import numpy as np

# Nur für die schönen Fortschrittsbalken, nicht von dem seltsamen Namen irritieren lassen.
from tqdm.notebook import tqdm

# Falls der Rechner eine Graphikkarte hat und die entsprechenden Bibliotheken
# installiert sind, rechnen wir auf der Graphikkarte. Sonst auf dem normalen Prozessor (*sehr* langsam).
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Als nächstes Definieren wir die Architektur unseres rekurrenten neuronalen Netzwerkes. Unser Netzwerk kann auch komplexer aussehen, als das obige Beispiel, zum Beispiel so:

<img src="./figure_network.png">

Das Netzwerk kann mehrere hidden Layers besitzen (in dem Beispiel im Bild 3), was es erlaubt, noch komplexere Input-Hidden-Output-Funktionen zu lernen ("num_layers"). Jedes dieser Layer besitzt "hidden_size" Neurone, in dem Bildbeispiel 512. Die Anzahl der Input- und Outputneurone hängt von dem Trainingsdatensatz ab. Sie entspricht der Anzahl der Buchstaben in den Trainingstexten ("# of chars").

In den folgenden Zelle definieren wir eine RNN "Klasse". Das ist eine Struktur, die Definiert, welche Daten und Funktionen ein rekurrentes neuronales Netzwerk besitzen sollte. Solche Klassen erlauben es später, konkrete "Instanzen", d.h. einzelne neuronale Netzwerke zu erstellen, die man auf bestimmten Daten trainieren, mit gespeicherten Gewichten initialisieren kann, und zum erzeugen von Text benutzen kann. Dabei muss man die konkreten Eigenschaften des RNNs erst beim erzeugen einer konkreten "Instanz" festlegen, d.h. z.B. wie viele Hidden Layers und wie viele Neurone pro hidden Layer das RNN haben soll. Schauen wir uns jetzt erstmal unsere RNN-Klasse an:

In [ ]:
# Hier definieren wir eine Klasse, die unseren eigenen rekurrenten neuronalen Netzwerken
# zu Grunde liegen wird.

class RNN(nn.Module):
    
    # Wenn wir eine Instanz, d.h. eine konkrete Realisierung, dieser Klasse erstellen, müssen wir die folgende
    # Information übergeben:
    # - num_chars: Wie viele Buchstaben hat das Alphabet unseres Trainingsdatensatzes. So viele
    # Neurone werden wir dem Input- und dem Output-Layer geben
    # - hidden_size: Wie viele Neurone soll jedes Hidden Layer haben
    # - num_layers: Wie viele hidden Layers soll unser Netzwerk haben
    # - char_to_ix: Übersetzungstabelle vom Trainingsalphabet zu numerischen Indizes (s. Abbildung unten)
    # - ix_to_char: Übersetzungstabelle von numerischen Indizes zum Trainingsalphabet (s. Abbildung unten)
    # - dropout: Parameter für Dropout-Regularisierung (mehr dazu unten im Absatz zu Overfitting)
    def __init__(self, num_chars, hidden_size, num_layers, char_to_ix, ix_to_char, dropout = 0.3):
        super(RNN, self).__init__()
        
        # Hier erzeugen wir unser Input-Layer, es *lernt* ein Embedding von num_chars diskreten Symbolen
        # (in unserem Fall: Buchstaben) in die Aktivität von num_chars Neuronen. 
        # Da das Embedding gelernt wird, könnte die Anzahl der Neurone im Input-Layer auch
        # größer oder kleiner sein, aber in der Praxis hat es sich bewährt,
        # die Anzahl der Neurone gleich der Anzahl der Symbole (Buchstaben im Alphabet)
        # zu wählen
        self.embedding = nn.Embedding(num_chars, num_chars)
        
        # Hier erzeugen wir unsere hidden Layers. Die input_size entspricht dabei der
        # Anzahl der Neuronen im Input Layer, die wir als die Zahl der Buchstaben im 
        # Trainingsalphabet gewählt haben.
        # Die Anzahl der Neuronen pro Hidden Layer (hidden_size) 
        # und die Anzahl der Hidden Layers (num_layers) kann (im Rahmen von Zeit und 
        # Arbeitsspeicherbeschränkungen) frei gewählt werden
        # Dropout bezeichnet eine Regularisierungsmethode, die dem Netzerk helfen soll,
        # robustere und besser zu verallgemeinernde Repräsentationen zu lernen.
        # dropout = 0.0 entspricht keiner Regularisierung
        # dropout = 0.3 entspricht einer starken Regularisierung
        # dropouts > 0.5 machen wenig Sinn, dropouts >= 1.0 funktionieren überhaupt nicht.
        self.rnn = nn.LSTM(input_size=num_chars, hidden_size=hidden_size, num_layers=num_layers, dropout=dropout)
        
        # Unser Output-Layer
        self.decoder = nn.Linear(hidden_size, num_chars)
        
        # Wir merken uns die ganzen Parameter und auch die Übersetzungstabellen
        self.hidden_size = hidden_size
        self.num_chars = num_chars
        self.num_layers = num_layers
        self.char_to_ix = char_to_ix
        self.ix_to_char = ix_to_char
    
    # Während die vorherige Funktion dazu diente, das Netzwerk zu konstruieren, wird diese Funktion
    # hier aufgerufen, um anhand eine Sequenz von Inputs (input_seq) und einem initialen 
    # Zustand der Hidden Layers (hidden_state) eine Sequenz von Aktivierungen der Hidden 
    # Layers und des Output-Layers zu berechnen.
    def forward(self, input_seq, hidden_state):
        
        # Die Inputs (Zahlen von 0 bis zu Anzahl der Buchstaben - 1)
        # werden in die Aktivität der Inputneurone Umgewandelt ("embedding")
        embedding = self.embedding(input_seq)
        
        # Mit dem initialen Hidden State und den Aktivitäten der Input-Neurone
        # werden die Aktivitäten der Hidden States berechnet
        hidden_output, hidden_state = self.rnn(embedding, hidden_state)
        
        # Aus der Aktivität des letzten Hidden Layers (nochmals etwas
        # verarbeitet, für Details s. https://colah.github.io/posts/2015-08-Understanding-LSTMs/)
        # wird die Aktivität des Output-Layers erzeugt
        output = self.decoder(hidden_output)
        
        # Die Outputs und die Hidden States werden Ausgegeben.
        # Die Indices [0], [1] hinter hidden_state weisen darauf hin,
        # dass LSTM-Netzwerke etwas komplizierter sind als in unserem obigen Beispiel.
        # Jedes LSTM-Neuron besitzt nämlich zwei Zustandsvariablen, die wir hier beide 
        # ausgeben.
        # Für Details zu LSTM s. https://colah.github.io/posts/2015-08-Understanding-LSTMs/
        return output, (hidden_state[0].detach(), hidden_state[1].detach())    

Die mysteriösen Strukturen char_to_ix und ix_to_char beinhalten Übersetzungstabellen zwischen den konkreten Buchstaben, die in den Trainingsdaten vorkommen, und den Zahlen von 0 bis zur Anzahl der Buchstaben - 1. Streng genommen sieht unser RNN nur diese Zahlen und lernt die Wahrscheinlichkeitsverteilung über die nächste Zahl, gegeben eine Folge von vorhergehenden Zahlen.

D.h. die Interpretation dieser Struktur als Texte erfordert diesen zusätzlichen Übersetzungsschritt. Deshalb ist es auch wichtig, sich diese Tabellen zu merken, wenn man ein vortrainiertes Netzwerk auf neue Daten anwenden bzw. zur Generierung neuer Texte nutzen möchte.

Hier noch ein Bild zur illustration anhand unseres Beispiels mit dem Alphabet aus vier Buchstaben:

<img src = "figure_translation.png" style="width:300px;"/>

Im folgenden bauen wir uns eine Funktion, der wir eine konkrete Instanz eines so ein vortrainiertes RNNs, zusammen mit einem Textstring, übergeben können, und welche diesen Textstring dann mit Hilfe des RNNs fortsetzt.

In [ ]:
# Zum Training kommen wir später, aber hier können wir uns einmal anschauen, wie genau man mit einem
# RNN Text erzeugt.

# Diese Funktion nimmt ein trainiertes RNN und erzeugt damit etwas Text.
# Sie bekommt dazu:
# - das trainierte RNN ("rnn"), 
# - das Gerät ("device"), auf dem das RNN laufen soll ('cuda' für Graphikkarte oder 'cpu' für CPU), 
# - eine vorgegebene Zeichenkette, die fortgesetzt werden soll ("prompt"), 
# - die Anzahl der Zeichen, die generiert werden sollen ("n").

def sample(rnn, device, prompt = '', n = 1000):

    # Wir können dem trainierten Netzwerk einen Starttext ("prompt") geben, von dem aus
    # es weiter Text erzeugen soll. Falls dieser nicht leer ist, geben wir ihn aus.
    if prompt != '':
        print('\nPrompt: ' + prompt + '\n')
    # Sonst initialisieren wir den Prompt mit einem Zeilenumbruch (Buchstabe '\n')
    else:
        prompt = '\n'
    
    # Hier sagen wir PyTorch, dass wir das RNN nicht trainieren wollen, d.h. wir müssen uns
    # keine Information merken, die wir nur dazu brauchen, später die Ableitung (den Gradienten "grad")
    # der Kostenfunktion nach den Netzwerkparametern zu bestimmen. 
    # Der Code zum Training kommt in der letzten Zelle ganz unten.
    
    with torch.no_grad():        
    
        # Damit unser RNN den Text versteht, übersetzen wir zuerst jedes Zeichen in einen numerischen
        # Index. Dieser wird während des Trainings anhand des Trainingstextes festgelegt, so dass alle
        # Zeichen im Trainingstext abgebildet werden. Umgekehrt heisst das, dass unser Netzwerk
        # nur Zeichen kennt, die auch im Trainingstext vorkamen. D.h. bei einem englischen Trainingstext würde
        # es z.B. später keine deutschen Umlaute kennen und man würde beim Ausführen eine
        # Fehlermeldung bekommen.
        
        # Wir machen aus der Zeichenkette eine Liste aus einzelnen Zeichen
        start_seq = list(prompt)
        
        # Wir zählen alle Elemente in dieser Liste durch
        for i in range(len(start_seq)):
            
            # Wir ersetzen das Zeichen an der Stelle "i" (ch)
            # durch den enstprechenden Index aus der Übersetzungs
            # Tabelle, die in rnn.char_to_ix[ch] gespeichert ist.
            ch = start_seq[i]
            start_seq[i] = rnn.char_to_ix[ch]

        # Wir schicken die Liste, die jetzt nur Zahlen zwischen 0 und der Anzahl der verschiedenen
        # Textzeichen, die unser Netzwerk kennt, minus 1 enthält, zur Graphikkarte.
        start_seq = torch.tensor(start_seq).to(device)
        
        # Wir initialisieren den "hidden state" unseres LSTM-Netzwerkes mit Nullen. Damit "Löschen"
        # wir das Kurzzeitgedächtnis des RNNs, d.h. dass es vor unserem "Prompt" noch 
        # keine weiteren Zeichen gesehen hat.
        
        # Zur Erinnerung: Wir haben hidden_size Neurone in jedem Hidden Layer, 
        # und jedes Layer ist ein LSTM, in dem jedes Neuron
        # *zwei* Zustandsvariablen besitzt.
        hidden_state =  ( 
                            torch.zeros((rnn.num_layers, 1, rnn.hidden_size)).to(device),
                            torch.zeros((rnn.num_layers, 1, rnn.hidden_size)).to(device)
                        )
                
        # Zunächst zeigen wir unserem Netzwerk den Prompt. Dazu iterieren wir über alle
        # Zeichen dieses Starttextes. Der Output des Netzwerkes gibt uns dabei für jedes 
        # Zeichen die Wahrscheinlichkeitsverteilung über das nächste Zeichen.
        # Das brauchen wir später, um neue Zeichen zu generieren. So lange wir aber nur den
        # bereits vorhandenen Text lesen, ist das wichtigste, dass wir dem Netzwerk immer
        # wieder den "hidden_state" zurückgeben, der mit jedem Zeichen aktualisiert wird und
        # mit dem sich das Netzwerk den Prompt in seinem Kurzzeitgedächtnis merkt,
        # um dann (hoffentlich) darauf aufbauend eine sinnvolle Fortsetzung dieses
        # Textes zu generieren.
        # Beachte: "prompt" ist die originale Zeichenkette, "start_seq" ist deren Übersetzung
        # in Zahlen, die schon an die Graphikkarte geschickt wurde. Beide sind gleich lang.
    
        for i in range(len(prompt)):
            output, hidden_state = rnn(start_seq[i].reshape(1,1), hidden_state)
            
        # Jetzt hat unser Netzwerk den "prompt" gelesen und in seinem hidden_state gespeichert.
        # Nun erzeugen wir "n" neue Zeichen. Wichtig ist dabei auch, dass wir immer wieder den
        # gleichen "hidden_state" an das Netzwerk zurückgeben, so dass es die neu hinzugekommenen
        # Zeichen seiner Kurzzeitgedächtnisrepräsentation (d.h. seinem hidden_state) hinzufügen kann.
        
        # Da wir in der kommenden Schleife die erzeugten Buchstaben nacheinander ausgeben, geben wir hier nochmal
        # den Prompt aus, ohne eine neue Zeile dahinter (end = ''), damit man sehen kann, wie der Prompt
        # genau fortgesetzt wird.
        
        print('\nGenerated text:\n')
        print(prompt,end = '')
            
        for i in range(n):
            
            # Die F.softmax-Funktion nimmt den letzten output des Netzwerkes aus dem letzten 
            # Durchlauf der vorhergehenden "Leseschleife" (für n=0) oder dem vorhergehenden
            # Durchlauf dieser Schleife.
            # Errinern sie sich daran, dass das RNN darauf trainiert wurde, anhand des gerade gezeigten
            # Zeichens und seines "hidden_state", der die zuvor gesehene Zeichenkette repräsentieren soll,
            # das nächste Zeichen vorherzusagen.
            # Dazu müssen die Aktivierungen des Output-Layers zunächst mit der Softmax-Funktion
            # in eine richtige Wahrscheinlichkeitsverteilung umgewandelt werden, so dass
            # 1. Buchstaben mit einer höheren Output-Aktivierung eine höhere Wahrscheinlichkeit bekommen
            # 2. alle Wahrscheinlichkeiten positiv sind
            # 3. sich die Wahrscheinlichkeiten zu 1.0 aufsummieren.
            # Details zu dieser Funktion folgen in der nächsten Zelle.
            softmax_output = F.softmax(torch.squeeze(output), dim=0)
            
            # Mit dieser Wahrscheinlichkeiten initialisieren wir eine "Categorical"-Struktur.
            # Diese Struktur beschreibt eine kategorielle Wahrscheinlichkeitsverteiltung,
            # d.h. eine Wahrscheinlichkeitsverteilung über verschiedene Klassen, Würfelseiten, Buchstaben, ...
            # Für uns ist hier wichtig, dass es diese Struktur erlaubt, ganz einfach Beispiele aus so einer
            # Verteilung zu ziehen (s. nächste Codezeilen).
            
            # Wir initialisieren die Verteilung mit den Wahrscheinlichkeiten, die wir gerade
            # aus unseren Outputs berechnet haben.
            dist = Categorical(softmax_output)
            
            # Hier nutzen wir die gerade Erzeugte Categorical Struktur
            # um ein zufälliges Beispiel aus der vom RNN (+Softmax) berechneten
            # Wahrscheinlichkeitsverteilung zu ziehen, also quasi mit einem "unfairen"
            # Würfel zu würfeln, der soviele Seiten hat wie es Buchstaben in unserem Alphabet gibt,
            # und der so gezinkt ist, dass die einzelnen Seiten die Wahrscheinlichkeiten haben,
            # die unser Softmax-Output beschreibt.
            next_index = dist.sample()
            
            # Diesen Index wandeln wir jetzt mit der Übersetzungstabelle des Netzwerkes in den 
            # nächsten Buchstaben um.
            next_char = rnn.ix_to_char[next_index.item()]
            
            # Diesen Buchstaben geben wir jetzt mit dem print-Befehl aus, ohne einen Zeilenumbruch zu
            # erzeugen (in dem wir den end='' Parameter übergeben).
            print(next_char, end='')
            
            # Den generierten Index zeigen wir jetzt unserem Netzwerk, um die Gedächtnisrepräsentation
            # zu aktualisieren und die Vorhersage für den nächsten Output zu erzeugen
            output, hidden_state = rnn(next_index.reshape(1,1), hidden_state)

### Kurzer Exkurs zur Softmax-Funktion

Die Funktion, die wir benutzen, um die Aktivierungen unser Output-Neurone ("Scores") in eine Wahrscheinlichkeitsverteilung umzuwandeln ist die sogenannte Softmax funktion.

Für einen Vektor $\vec{\hat{y}} = (\hat{y}_1, \hat{y}_2, ..., \hat{y}_n)$, der die Aktivierung von $n$ Outputneuronen enthält (ein Neuron für jeden der $n$ Buchstaben im Alphabet des Trainingstextes), berechnen sich die entsprechenden Wahrscheinlichkeiten $\vec{p} = (p_1, p_2, ..., p_n)$ als:

$$ \vec{p} = \mathrm{Softmax}(\vec{\hat{y}}) $$

mit

$$ p_i = \frac{e^{\hat{y}_i}}{\sum_{j=1}^n e^{\hat{y}_j}} $$

In der folgenden Zelle können sie ausprobieren, was diese Funktion mit einem kurzen Vektor von Output-Aktivierungen macht. Probieren sie hier gerne verschiedene (hohe/niedrige, stark unterschiedliche/kaum unterschiedliche) Werte der Aktivierungen aus.

In [ ]:
from RNN_helper_functions import softmax_demo

# Output-Vektor, der in eine Wahrscheinlichkeitsverteilung umgewandelt wird

demo_output = [0.2, -1.5, -0.1, 2.2]

softmax_demo(demo_output)

# Zurück zu unserem RNN und den Texten

Um uns etwas an die Funktionsweise des RNNs zu gewöhnen, laden wir zunächst ein vortrainiertes RNN, welches auf einer Rezeptdatenbank trainiert wurde.

Die Textdatei, mit der trainiert wurde, finden sie in diesem Ordner im Unterordner **"data"** mit dem Namen **"recipes.txt"**. Schauen sie gerne einmal rein und achten sie darauf, welche Struktur die einzelnen Rezepte haben.

In [ ]:
# Wir laden zunächst die beiden Übersetzungstabellen. Der Datensatz hatte 99 Zeichen

with open('preTrained/char_to_ix_recipes_demo.pkl','rb') as file:
    char_to_ix = pickle.load(file)
    
with open('preTrained/ix_to_char_recipes_demo.pkl','rb') as file:
    ix_to_char = pickle.load(file)
    
# Schauen wir uns eine der Tabellen mal an (von Indices nach Buchstaben):
print(ix_to_char)

In [ ]:
# Wie bereits erwähnt hatte der Trainingsdatensatz 99 verschiedene Zeichen, 
# Außerdem hat das trainierte RNN 3 LSTM-Layer mit jeweils 512 Neuronen.
# Deshalb müssen wir hier die entsprechenden Parameter angeben, wenn wir unser "test_rnn"
# mit Hilfe der RNN-Klasse (s. oben) erzeugen. 
    
test_rnn =  RNN(99, 512, 3, char_to_ix, ix_to_char, dropout = 0.3).to(device)

# Nun können wir die vortrainierten Netzwerkgewichte aus der entsprechenden Datei laden.
test_rnn.load_state_dict(torch.load('./preTrained/CharRNN_recipes_demo.pth'))

# Endlich können wir ein paar Zeichen mit dem trainierten Netzwerk erzeugen.
sample(test_rnn, device)                                  

Das sieht doch schon gar nicht so schlecht aus. 
Wir können dem Netzwerk auch einen Anfangstext vorgeben und die Länge der generierten Textsequenz bestimmen.

In [ ]:
sample(test_rnn, device, prompt = 'Title: CHEESECAKE', n = 2000)

# Wie trainiert man ein RNN auf eigenen Daten?

**Mindestens** so wichtig wie die Architektur des neuronalen Netzwerkes und die Wahl der Zielfunktion ist auch die Auswahl und Vorverarbeitung der Trainingsdaten, denn ein tiefes neuronales Netzwerk kann nur die statistische Struktur der Trainingsdaten lernen, die man ihm anbietet. Und - falls das Training klappt - lernt es eben die komplette Struktur in den Daten, nicht nur die Zusammenhänge, auf die man als Nutzer\*in abzielt. D.h. das Netzwerk lernt alles an Struktur zu Nutzen, um seine Zielfunktion zu optimieren, was ihm die Trainingsdaten anbieten, **inklusive zufälliger Korrelationen und Vorurteile (Dataset Biases)** die in diesen Daten enthalten sind.

In [ ]:
# Wir starten damit, dass wir unserem Modell einen Namen geben
# Unter diesem Namen werden später die trainierten Gewichte und
# die Übersetzungstabellen zwischen Buchstaben und Netzwerk In-
# bzw. Outputs gespeichert, sowie die genaue Zusammensetzung
# der Trainingsdaten

model_name = 'my_own_recipes'

# Hier wird aus dem Modell-Namen der genaue Pfad zusammengesetzt, unter dem die
# trainierten Gewichte während des Trainings gesichert werden bzw. wurden.
save_path = './CharRNN_' + model_name + '.pth'

# Vorbereitung der Trainingsdaten

In [ ]:
# Als erstes legen wir den Trainingsdatensatz fest.
# Hierbei muss es sich nur um eine - möglichst Lange - Textdatei
# handeln, deren Struktur das RNN lernen soll.
# Experimentieren sie auch gerne mit anderen Datensätzen und Dateien.
# Sie können dazu gerne eigene Textdateien auf Jupyter hochladen oder direkt
# in Jupyter ein neues Textdokument erstellen und Text dort hinein kopieren.
data_path = './data/recipes.txt'

Zunächst laden wir die Trainingsdaten aus der Textdatei. Dann geben wir die Länge der Trainingsdaten und die Anzahl der unterschiedlichen Zeichen in den Daten aus

In [ ]:
# Wir lesen den Inhalt der Datei
data = open(data_path, 'r').read()
# Wir generieren eine Liste mit dem Alphabet an verschiedenen Buchstaben, die in den Trainingsdaten vorkommen.
chars = sorted(list(set(data)))
# Wir zählen die gesamte Anzahl der Zeichen im Trainingsdatensatz, und die Länge des Alphabets.
data_size, num_chars = len(data), len(chars)

# Und wir geben die Längen aus.
print("----------------------------------------")
print("Data has {} characters, {} unique".format(data_size, num_chars))
print("----------------------------------------")

Wir erstellen zwei Übersetzungstabellen, um die Buchstaben in eine aufsteigende Zahlenfolge zu übersetzen (char_to_ix) und umgekehrt (ix_to_char). Das hat den Hintergrund, dass die PyTorch-Funktion, mit der wir das Embedding der Inputs durchführen, aufsteigende Zahlenreihen (von 0 bis zur Anzahl der verschiedenen Buchstaben) erwartet.

In [ ]:
# Die Details der nächsten zwei Zeilen sind nicht wichtig,
# im Prinzip werden zwei Übersetzungstabellen erstellt.
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

# Wir speichern die Übersetzungstabellen dieses Datensatzes, falls wir das trainierte RNN
# später ohne die Trainingsdaten benutzen oder weitergeben wollen:
with open('char_to_ix_' + model_name + '.pkl', 'wb') as output:    
    pickle.dump(char_to_ix, output)
with open('ix_to_char_' + model_name + '.pkl', 'wb') as output:    
    pickle.dump(ix_to_char, output)

# Wir geben noch die Übersetzungstabelle von den Indices (Zahlen) zu den Buchstaben aus, auch
# um zu sehen, welche Zeichen in unseren Trainingsdaten vorkommen
print("Extracted the following characters:")
print(ix_to_char)
print("----------------------------------------")

Nun übersetzen wir die eingelesenen Textdaten in die entsprechenden Indizes (Zahlen) und schicken die Zahlenreihe an die Graphikkarte.

In [ ]:
# convert data from chars to indices
data = list(data)
for i, ch in enumerate(data):
    data[i] = char_to_ix[ch]

# data tensor on device
data = torch.tensor(data).to(device)
data = torch.unsqueeze(data, dim=1)

### Trainings-, Test- und Validierungsset

Ein großes Problem an Modellen, die Millionen an freien Parametern haben, ist das sogenannte Overfitting. Hierbei ist das Modell so flexibel, dass es nicht nur die Regularitäten bzw. die Struktur in den Daten aufgreift, sondern auch das __zufällige Rauschen in den Daten lernt__. Das führt natürlich dazu, dass solch ein zu flexibles Modell schlecht generalisiert, d.h. schlecht neue Datenpunkte, die ein anderes zufälliges Rauschen mitbringen, vorhersagt.

In der nächsten Zelle können sie ausprobieren, einen Datensatz (blaue Punkte), der auf Grundlage einer deterministischen Funktion (orange) mit additivem, zufälligen Rauschen generiert wurde, durch ein Polynom anzupassen. Dabei gibt der Grad des Polynoms an, wie viele freie Parameter angepasst werden können. Ein Grad von eins entspricht dabei einer Geraden mit zwei freien Parametern (Steigung und y-Achsenabschnitt). Ein Grad von zwei entspricht einer quadratischen Funktion mit drei freien Parametern, usw...

In der folgenden Zelle können sie der Funktion __plot_overfitting_demo__ eine Liste mit Graden (1,2,3,...) übergeben, für die jeweils ein Polynom des entsprechenden Grades an die Stichprobe gefittet wird. Die entsprechenden Graphen werden nebeneinander dargestellt (ab 4 wird es ziemlich gequetscht).

Probieren sie gerne einmal aus, wie sich die Annäherung an die Daten verändert, wenn man von sehr einfachen Funktionen (Grad 1) zu __sehr, sehr komplizierten (Grad 100) geht__. Beachten sie dabei, dass mit zunehmendem Grad der mittlere quadratische Fehler auf den Trainingsdaten immer kleiner wird, die Annäherung der __wahren Funktion (orange)__ durch das __Modell ("Polynom", blau)__ aber nicht unbedingt besser und der Fehler auf neuen, ungesehenen Daten auch nicht unbedingt kleiner wird.

In [ ]:
from RNN_helper_functions import plot_overfitting_demo

# Gerne auch mehrere Grade (>= 1) in die Liste eintragen, zum vergleichen
grade = [1,3,15] 

plot_overfitting_demo(grade)

In der Praxis hat sich im Deep Learning folgendes Vorgehen durchgesetzt, um Overfitting zu vermeiden:

Man teilt die Daten, mit denen der Algorithmus trainiert wird in drei __unabhängige__ Teilmengen:

- Ein __Trainingsset__: Auf dem Trainingsset wird der Gradient der Kostenfunktion berechnet, der von dem gradientenbasierten Optimierungsalgorithmus genutzt wird, um die Parameter der neuronalen Netze zu optimieren.

- Ein __Validierungsset__: Auf dem Validierungsset wird während des gradientenbasierten Trainings immer wieder die Kostenfunktion ausgewertet. So lange das neuronale Netz noch sinnvolle Struktur aus dem Trainingsset lernt, sollte auch die Kostenfunktion auf dem Validierungsset fallen. Sobald während des Trainings jedoch der Wert der Kostenfunktion auf dem Validierungsset zu steigen beginnt, während er auf dem Trainingsset weiter fällt, geht man davon aus, dass das Netz beginnt zu "überfitten" und beendet das Training. Dies wird in der Literatur als __early stopping__ bezeichnet.

- Ein __Testset__: Das Testset wird benutzt, um die finale Performance des trainierten Netzwerkes zu überprüfen. Es muss vom Trainings- und Validierungsset möglichst komplett unabhängig sein.

Bevor wir also mit dem Training unseres Netzes beginnen können, teilen wir die Trainingsdaten in **drei nicht überlappende Teildatensätze** ein.

Üblich sind z.B. ein Split von 80% der gesamten Trainingsdaten als Trainingsset, und je 10% als Validierungs- und Testset.

Wählen sie auch gerne eine andere Aufteilung, z.B. 60%/20%/20%.

Idealer Weise wären diese drei Datensätze aus unterschiedlichen Quellen, z.B. Artikel von unterschiedlichen News-Seite, unterschiedliche Bücher, ...

Der Einfachheit halber teilen wir jedoch unsere monolithischen Daten einfach in drei Teile. Das ist alles andere als optimal, sollte es für diese Beispiele aber tun.

In [ ]:
# Floor rundet eine Zahl ab, also 1.3 -> 1.0, 2.8 -> 2.0
from numpy import floor

# Kleine Helferfunktion (s.u.)
from RNN_helper_functions import save_dataset_info

# Prozentaler Anteil der Daten, die ins Trainingsset kommen
train_percentage = 0.8
# Prozentualer Anteil der Daten, die ins Validierungsset kommen
valid_percentage = 0.1
# Der Rest kommt ins Testset
test_percentage = 1.0 - train_percentage - valid_percentage

# Die Indices der Zeichen im gesamten Datensatz, die die Grenze von
# Trainings-, Validierungs- und Testset bilden.
# Es muss sich hier um ganze Zahlen handeln, deshalb wird abgerundet mit floor()
valid_index_start = int(floor(train_percentage*data_size))
test_index_start = int(floor((train_percentage + valid_percentage)*data_size))

# Wir schneiden die ersten train_percentage Prozent der gesamten
# Daten als Trainingsdaten ab.
train_data = data[0:(valid_index_start-1)]
# Wir schneiden die darauf folgenden valid_percentage 
# Prozent der gesamten Daten als Validierungsdaten ab.
valid_data = data[valid_index_start:(test_index_start-1)]
# Die übrigen Daten packen wir ins Testset.
test_data = data[test_index_start:]

# Wir speichern den Dateinamen der Textdatei, die die gesamten Daten enthält, und
# die Indices, die wir benutzt haben, um unsere Daten aufzuteilen, damit wir
# später genau rekonstruieren können, was unser Trainings-, Validierungs- und Testset war.
save_dataset_info(data_path, [valid_index_start, test_index_start], 'dataset_' + model_name + '.txt')

# Hier berechnen wir die Länge (Anzahl Buchstaben) der einzelnen
# Datensätze und geben diese aus.
train_data_size = len(train_data)
valid_data_size = len(valid_data)
test_data_size = len(test_data)
print('Size of training dataset: %d' % train_data_size)
print('Size of validation dataset: %d' % valid_data_size)
print('Size of test dataset: %d' % test_data_size)

Hier definieren wir uns eine Trainingsfunktion, die ein RNN auf einer großen Textdatei trainiert. Wie oben bereits besprochen ist
das Ziel des RNNs die Wahrscheinlichkeitsverteilung über das jeweils nächste Zeichen in einer gegebenen Zeichenfolge zu lernen. Rufen wir uns das Beispiel-RNN vom Anfang nochmal in Erinnerung:

<img src="./charseq.png">

Dieses RNN wurde auf einem Datensatz mit nur vier Buchstaben ('h','e','l','o') trainiert. Dementsprechend besitzt das RNN auch vier Output-Units, die jeweils einen Score für jeden Buchstaben im trainierten Alphabet angibt. Dieser Score wird mittels der Softmax-Funktion in die Wahrscheinlichkeit, dass der entsprechende Buchstabe als nächstes in der Zeichenkette folgt umgewandelt (höher = größere Wahrscheinlichkeit). Weiterhin hat das RNN drei Neuronen in seinem "hidden layer". Dieses tolle Diagramm (aus einem großartigen Blogpost) zeigt die Aktivierungen der Input, Hidden, und Output Neurone während dem Netzwerk nacheinander die Buchstaben der Sequenz "hell" präsentiert werden. Die entsprechenden Output-Aktivierungen geben dabei jeweils einen Score an, der quantifiziert, für wie wahrscheinlich das Netzwerk es hält, dass der entsprechende Buchstabe als nächster Buchstabe in der Sequenz folgt. Die Scores der tatsächlich folgenden Buchstaben, die nochmal in der Sequenz "target" gezeigt werden, wurden dabei grün eingefärbt. Ziel des Netzwerktrainings ist es, die (grünen) Scores der tatsächlich folgenden Buchstatben so hoch wie möglich zu bekommen und die der nicht folgenden Buchstaben (rot) so niedrig wie möglich.

Beachten sie, dass dem Netzwerk hier zweimal der Buchstabe 'l' präsentiert wird. Auf das erste 'l' folgt wieder ein 'l', während auf das zweite 'l' ein 'o' folgt. D.h. der Output des Netzwerks kann nicht nur von seinem direkten Input abhängen, damit es die Vorhersage solcher Zeichenfolgen richt lernen kann. Das ist die große Stärke von RNNs und anderen Methoden (z.B. Transformern, mit denen wir uns im nächsten Workshop beschäftigen werden), die den "Kontext" eines Inputs miteinbeziehen können.

# Definition der Netzwerkarchitektur

Nun legen wir die Architektur unseres Netzwerkes fest.
Die zwei der drei freien Parameter sind dabei die Anzahl der Neuronen
pro Netzwerkebene ("hidden_size") und
die Anzahl dieser Ebenen ("num_layers"). Mit drei Hidden Layers und 512 Neuronen pro Hidden Layer erhält man z.B. die folgende Architektur: <img src="./figure_network.png">

In [ ]:
hidden_size = 512   # size of hidden state
num_layers = 3      # num of layers in LSTM layer stack

### Dropout-Parameter

Eine weitere Methode, um die Generalisierung von trainierten Netzwerken auf neue Daten zu verbessern ist das sogenannte Dropout. Dabei wird ein gewissen Prozentsatz der Neurone während des Trainings zufällig auf "0" gesetzt. Dieses Rauschen im Trainingsprozess zwingt das Netzwerk dazu, robustere Repräsentationen zu lernen, und nicht kleine (oft zufällige) Details im Trainingsdatensatz auswendig zu lernen.

Übliche Werte für den Dropout Parameter sind 0.0 - 0.5 (wobei 0.5 bedeutet, dass ca. die **Hälfte** der Neuronen in jedem Trainingsdurchgang zufällig genullt werden).

In [ ]:
# Der Prozentsatz der Gewichte, die bei jedem Durchlauf zufällig auf Null
# gesetzt werden sollen ist in der Variable "dropout" gespeichert.
# Beachte: 0.1 entspricht hier 10%.
dropout = 0.3

Als nächstes Erstellen wir ein RNN mit den von uns weiter oben gewählten Parametern, und der passenden Anzahl an Inputs und Outputs (die Anzahl der verschiedenen Buchstaben bzw. Indices).

Außerdem speichern wir die gewählten Parameter in einer Textdatei. Zum einen zur besseren Reproduzierbarkeit, aber auch um später unser Netzwerk in anderen Notebooks benutzen zu können.

Falls wir mit vortrainierten Gewichten starten wollen, laden wir noch die entsprechende Datei.

In [ ]:
from RNN_helper_functions import save_model_parameters

# Erzeuge eine konkrete Instanz der Modellklasse, mit den von Ihnen gewählten Parametern
rnn = RNN(num_chars, hidden_size, num_layers, char_to_ix, ix_to_char, dropout).to(device)

# Speichere die Modellparameter unter dem von Ihnen gewählten Modellnamen ab.
save_model_parameters(model_name, hidden_size, num_layers, num_chars, dropout)

# Wenn es die entsprechenden Gewichte schon gibt, kann man statt mit einem
# zufällig initialisierten Modell, die vortrainierten Gewichte laden und damit
# das Training fortsetzen, in dem man loach_chk auf True setzt
load_chk = False    # load weights from save_path directory to continue training

# load checkpoint if True
if load_chk:
    rnn.load_state_dict(torch.load(save_path))
    print("Model loaded successfully !!")
    print("----------------------------------------")

# Definition der Zielfunktion

Nun geht es langsam ans Eingemachte. Als nächstes definieren wir die Zielfunktion, die unser Netzwerk auf den Trainingsdaten minimieren soll, d.h. für die später der entsprechende Gradient für jedes Trainings-Batch (n_heads Sequenzen der Länge seq_len) berechnet wird.

In [ ]:
# loss function and optimizer
loss_fn = nn.CrossEntropyLoss()

Wir benutzen hier das CrossEntropyLoss, das klingt kompliziert, aber im wesentlichen berechnet es für jedes Zeichen des Trainings-Batches die Wahrscheinlichkeit für das nächste Zeichen, die unser Netzwerk mit Hilfe seines entsprechenden "hidden_state" vorhersagt und mittelt diese. Dabei entspricht ein niedrigeres Cross-Entropy-Loss einer höheren Wahrscheinlichkeit, d.h. einer besseren Vorhersage durch das Netzwerk.

Das klingt zwar sehr kompliziert, aber im wesentlichen funktioniert diese Kostenfunktion so:

- Jede Klasse erhält ein Output-Neuron
- Für jedes Trainingsbeispiel werden die Aktivierungen der Output-Neuronen in eine Wahrscheinlichkeitsverteilung über die einzelnen Klassen umgerechnet, so dass eine höhere Aktivierung eines bestimmten Output-Neurons einer höheren Wahrscheinlichkeit der entsprechenden Klasse entspricht. Dies wird in der Praxis gemacht, in dem man die Aktivierungen der Output-Neurone durch eine sogenannte SoftMax-Funktion schickt.
- Die Kreuzentropie für einen Satz (i.e. ein Batch oder einen Stapel) Trainingsbeispiele ist dann die __mittlere Wahrscheinlichkeit der tatsächlichen, wahren Klasse unter der von Netz vorhergesagten Wahrscheinlichkeitsverteilung__. D.h. diese Zielfunktion sorgt dafür, dass die Wahrscheinlichkeit, die das Netz der tatsächlichen Klasse eines Trainingsbeispiels gibt, immer größer wird. Oder anders gesagt: Dass die Aktivität des Output-Neurons, dass die wahre Klasse eines Trainingsbeispiels repräsentiert, größer wird.

Eine ausführliche Beschreibung dieses Zusammenhanges finden sie z.B. in diesem tollen [Blogartikel](https://towardsdatascience.com/cross-entropy-for-classification-d98e7f974451).

Zum Glück bring PyTorch (wie auch TensorFlow) die meisten gebräuchlichen Kostenfunktionen schon mit, so dass es reicht, die entsprechende Struktur aus PyTorchs NeuralNetwork ("nn") Bibliothek zu benutzen.

# Trainingsparameter

Nun legen wir die sogenannten "Hyperparameter" fest, die beschreiben, *wie* wir dieses Netzwerk trainieren.

### Lernrate

Wie in der Vorlesung besprochen, ist bei gradientenbasierten Optimierungsverfahren die Wahl einer passenden "Schrittweite" in Richtung der Gradienten der Zielfunktion wichtig. Ist die Schrittweite zu klein, dauert es mitunter sehr, sehr lange, bis das Netzwerk eine gute Performance erreicht. Ist die Schrittweite zu groß, kann es tatsächlich sein, dass es überhaupt nichts sinnvolles lernt, da es immer wieder aus den entsprechenden lokalen Minima der Zielfunktion herausspringt.

Als kleinen Exkurs können sie in der nächsten Zelle mit einer Funktion spielen, die einen Gradientenabstieg auf einer eindimensionalen, quadratischen Zielfunktion simuliert. D.h. es gibt nur einen einzigen, skalaren Parameter $w$, und die Zielfunktion ist eine quadratische Funktion dieses Parameters. Sie können mit __w_start__ bestimmen, von welchem Parameter-Wert der Gradientenabstieg starten soll. Zudem können sie die Lernrate __learning_rate__ bestimmen, die skaliert, wie groß die Optimierungsschritte in Richtung des Minimums sein sollen. Außerdem können sie mit __n_steps__ angeben, wie viele Optimierungsschritte gemacht werden sollen.

Als Output erhalten sie zwei Graphen. Der Linke zeigt die Zielfunktion (blau) als Funktion des Parameters $w$, und wie sich die Parameter-Werte mit jedem Optimierungsschritt vom Startwert (dunkelblauer Marker) zum finalen Wert (roter Marker) ändern (orangene Pfeile).

Rechts sehen sie den Wert der Kostenfunktion als Funktion des Optimierungsschrittes. Ein funktionierender Optimiser sollte am Schluss ziemlich nahe am Minimum __0__ der Kostenfunktion sein.

Spielen sie etwas mit den Parametern. Beobachten sie dabei, was passiert wenn sie die Lernrate sehr klein (z.B. 0.01) oder sehr groß (z.B. 2.0) machen. Schauen sie, wie schnell der Algorithmus für sinnvolle Werte (in diesem Beispiel z.B. 0.3) der Lernrate zum Minimum konvergiert.

In [ ]:
from RNN_helper_functions import simulate_gradient_descent_on_quadratic_potential

# Lernrate (probieren sie z.B. 0.02, 0.05, 0.1, 0.2, 0.3, 0.6, 0.9, 1.0, 1.1)
learning_rate = 0.3


# Anzahl der simulierten Schritte
n_steps = 10
# Startwert für den Parameter theta
w_start = -2.0
simulate_gradient_descent_on_quadratic_potential(w_start, learning_rate, n_steps)

Wie sie sehen, hat die Wahl der Lernrate einen kritischen Einfluss auf das Konvergenzverhalten von gradientenbasierten Optimierungsverfahren. Umso unbefriedigender ist es, dass es noch keine wirklich fundierte Methode gibt, diese festzulegen. In der Praxis ist meistens eine Menge Ausprobieren involviert, in dem man z.B. numerische "Experimente" durchführt, die einen ganzen Bereich von Lernraten ausprobieren.

Üblicher Weise funktionieren Lernraten im Bereich von 0.00001 - 0.001.

In der folgenden Zellen haben sie neben der Lernrate auch die Wahl des gradientenbasierten Optimierungsverfahrens. Wir bieten ihnen hier zwei Verfahren an:
- SGD oder "stochastic gradient descent" ist der absolute Klassiker, der tatsächlich genau das macht, was wir Ihnen in der Vorlesung gesagt haben. Also: Den Gradienten der Zielfunktion bestimmen und dann einen Schritt entgegen diesem Gradienten (erinnern sie sich, dass der Gradient einer Funktion die Richtung des __steilsten Anstieges__ ist) gehen, der mit der Lernrate skaliert wird. Dieser Algorithmus konvergiert bei genügend kleinen Lernraten fast immer und findet Minima, die gut generalisieren. Er ist jedoch auch sehr __langsam__. In der erweiterten Version, die wir hier benutzen gibt es noch einen Parameter, __"momentum"__, der dem Optimierungsprozess eine gewisse __Trägheit__ verleiht. Stellen sie sich eine schwere Metallkugel vor, die einen bergigen Hang hinunterrollt. Wenn diese auf eine kleine Mulde trifft, wird sie nicht in diesem lokalen Minimum hängen bleiben, sondern aufgrund ihrer Trägheit darüber hinwegrollen. 
- ADAM oder "adaptive moment estimation" hat sich seit seiner [Publikation](https://arxiv.org/pdf/1412.6980.pdf) 2014 zum "Schweizer Taschenmesser" der gradientenbasierten Optimierungsverfahren entwickelt. Das liegt daran, dass dieser Algorithmus, der seine Schrittweite während des Optimierungsprozesses anpasst, mit seinen Standardparametern über viele Anwendungen, Netzwerkarchitekturen und Zielfunktionen hinweg nicht immer die beste, aber immer eine passable Leistung bietet.

In [ ]:
# Lernrate
lr = 0.0003

# Gradientenbasiertes Optimierungsverfahren (kommentieren sie die 
# entsprechende Zeile ein bzw. aus)
optimizer = torch.optim.Adam(rnn.parameters(), lr=lr)
#optimizer = torch.optim.SGD(rnn.parameters(), lr=lr, momentum=0.9)

Ein weiterer wichtiger Hyperparameter beim trainieren von neuronalen Netzen ist die größe der Trainings-Minibatches. Diese bestimmt, wie viele Trainingsbeispiele in jedem Trainingsschritt benutzt werden, um den Gradienten der Kostenfunktion nach den Netzwerkgewichten zu berechnen. Im Falle unseres Textmodells, ist dieser Parameter gleichzeitig die Zahl der "Leseköpfe". Hierbei wandert jeder Lesekopf (ausgehend von einer zufälligen Startposition) durch das Dokument. Dabei extrahiert jeder Lesekopf für jeden Trainingsdurchlauf eine Zeichensequenz von gegebener Länge. Die entsprechenden Sequenzen werden nun benutzt um die Zielfunktion und deren Gradienten für den nächsten Trainingsschritt zu berechnen. Wie sie sich sicher denken können, ist die Länge der einzelnen Sequenzen auch ein wichtiger Hyperparameter, da er die Anzahl der zurückliegenden Zeitschritte bestimmt, die die Netzwerkgewichte noch beeinflussen können. Die Sequenzlänge bestimmt also quasi die "zeitliche Tiefe" unseres Netzwerktrainings. Ähnlich wie mit 
 anderen tiefen Neuronalen Netzen zeigt der Bedarf an Trainingszeit und
 Arbeitsspeicher deutlich mit der Tiefe des Netzwerkes. 
 Für die Graphikkarten die wir benutzen, und die Zeit, die wir heute haben,
 ist eine Lenge der Trainingssequenz von 100 Zeitschritte ein guter Ausgangspunkt. Aber experimentieren sie gerne
 mit zeitlich tieferem bzw. weniger tiefem Training.

In [ ]:
# Anzahl der Leseköpfe, deshalb heißt der entsprechende Parameter "num_heads"
num_heads = 16      

# Die Länge der Trainingssequenz, die jeder Lesekopf pro Trainingsiteration aus dem
# Trainingstext liest
seq_len = 100       

Nun bleiben noch ein paar Parameter übrig, die bestimmen, wie lange wir ingesamt (maximal) Trainieren möchten. Allerdings sollte man den Endzeitpunkt des Trainings natürlich am besten aus dem Kurvenverlauf der Zielfunktion auf dem Validierungsdatensatz ablesen (Stichworte: Overfitting und Early Stopping).

In [ ]:
# Für wie viele Epochen wollen wir trainieren? Eine Epoche bedeutet dabei, dass jeder Lesekopf
# einmal den gesamten Trainingstext durchlaufen hat.
epochs = 100

# Um das Training zu überwache, generieren wir alle sample_every Trainingsschritte
# einmal etwas Text der Länge sample_length, den wir mit sampling_prompt initialisieren.
sample_every = 1000  # sample every n-th training step        
sample_length = 200   # total num of characters in output test sequence
sampling_prompt = '''MMMMM----- Recipe via Meal-Master (tm) v8.05

  Title: MAC N CHEESE'''

# Außerdem berechnen wir alle sample_every Schritte den Wert der Zielfunktion
# auf dem Validierungsset und schreiben diesen zusammen mit Wert der Zielfunktion
# auf dem aktuellen Trainingsbatch in eine Log-Datei.
# Zudem speichern wir die aktuellen Netzwerkgewichte, so dass wir das Netzwerk
# auch in anderen Notebooks benutzen können.
log_every = 100

# Die Trainingsschleife

In der folgenden Zelle implementieren wir die Trainingsschleife, die in jedem Durchlauf für jeden Lesekopf eine Textsequenz extrahiert, diese Textsequenzen sammelt und dann damit die Netzwerkoutputs, die Kostenfunktion und den Gradienten der Kostenfunktion berechnet. In jedem Durchlauf der Schleife benutzt der Optimizer diese Gradienteninformation um die Netzwerkgewichte - entsprechend der Gewählten Lernrate - ein kleines Stück entgegen dieses Gradienten (d.h. in die Richtung, die die Kostenfunktion kleiner macht) zu bewegen.

In [ ]:
# Ein paar Hilfsfunktionen
from RNN_helper_functions import log_metrics_to_file, calculate_loss_on_dataset

# Hiermit zählen wir die Anzahl der Trainingsschritte, die wir bereits durchgeführt haben,
# damit wir in regelmäßigen Intervallen etwas Beispieltext generieren können.
steps_done = 0

# Wir berechnen die Anzahl der Trainingsschritte, die eine Epoche (d.h. ein kompletter)
# Durchlauf durch die Trainingsdaten hat
iter_per_epoch = int(train_data_size / (seq_len + 1))

# In dieser Variablen speichern wir die Trainingsdaten (input_seq)
input_seq = torch.zeros((seq_len, num_heads),dtype = data.dtype).to(device)

# Hier speichern wir die Zieldaten, die unser RNN voraussagt. Später
# werden wir diese Variable einfach mit der um einen Zeitschritt verschobenen
# Trainingssequenz füllen, da das Ziel des Trainings ja ist, den nächsten Buchstaben
# vorherzusagen.
target_seq = torch.zeros((seq_len, num_heads),dtype = data.dtype).to(device)

# Wir iterieren über die oben ausgewählte Anzahl an Epochen 
# (d.h. Durchläufen durch den gesamten Trainingsdatensatz)
for i_epoch in range(1, epochs+1):

    # Hier initialisieren wir die Poisition unserer num_heads Leseköpfe zufällig
    # für jeden Lesekopf. Wir müssen nur aufpassen, dass kein Lesekopf näher als
    # seq_len am Ende der Trainingsdaten beginnt,
    # da wir ausgehend von der Position der Leseköpfe immer die nächsten seq_len
    # Zeichen als Trainingsbatch extrahieren werden
    data_ptr = np.random.randint(train_data_size - seq_len - 1, size = num_heads)

    # Wir speichern hier die Summe der Zielfunktion, um den Mittelwert
    # über eine Epoche zu berechnen
    running_loss = 0
    
    # Wir initialisieren den hidden_state wieder bei Null, 
    # d.h. das Netzwerk startet mit einem "leeren Kurzzeitgedächtnis"
    hidden_state = ( 
                     torch.zeros((num_layers, num_heads, hidden_size)).to(device),
                     torch.zeros((num_layers, num_heads, hidden_size)).to(device)
                   )

    # Diese Schleife macht einen Schritt pro Durchlauf.
    # Das tqdm( ... ) ist nur dafür da, einen Forschrittsbalken zu bekommen, 
    # von der Logik her ist die nächste Zeile identisch zu
    # for i in range(iter_per_epoch):
    # D.h. i zählt von 0 bis zu Anzahl der Epochen, für die wir trainieren
    # wollen minus eins hoch.
    for i in tqdm(range(iter_per_epoch)):

        # Wir gehen alle Leseköpfe durch und speichern die entsprechenden
        # Zeichenketten von der momentanen Lesekopfposition data_ptr[j]
        # bis data_ptr[j]+seq_len in einem entsprechenden Eintrag
        # der input_seq Variablen.
        # Die target_seq beinhaltet die gleichen Sequenzen um eins in die Zukunft verschoben.
        # Dies ist das "Ziel", das vom RNN vorausgesagt werden soll (d.h. jeweils der nächste
        # Buchstabe in der Zeichenkette).
        for j in range(num_heads):
            input_seq[:,j] = (train_data[data_ptr[j] : data_ptr[j]+seq_len]).squeeze()
            target_seq[:,j] = (train_data[data_ptr[j]+1 : data_ptr[j]+seq_len+1]).squeeze() 

        # Da unsere Trainingsvariable bereits ganze Sequenzen enthält, müssen wir hier nichtmehr über die
        # Sequenz iterieren, wie wir es oben in der sample(...)-Funktion getan haben. 
        # PyTorch erledigt das für uns automatisch.
        output, hidden_state = rnn(input_seq, hidden_state)
        
        # Der Output enthält für jedes Zeichen jeder Inputsequenz eine Vorhersage
        # (in Form der Aktivität eines Output neurons für jeden Möglichen Buchstaben)
        # des jeweils *nächsten* Zeichens.
        
        # Diese Aktivitäten werden von der loss-Funktion automatisch in richtige Wahrscheinlich-
        # keitsverteilungen (alle Wahrscheinlichkeiten positiv und deren Summe gleich 1.0)
        # umgewandelt und dann wird für jede dieser Vorhersagen die Wahrscheinlichkeit
        # des *tatsächlich* folgenden Zeichens ausgewertet und davon jeweils der negative 
        # Logarithmus berechnet. D.h. je kleiner diese Zielfunktion desto höher ist die Wahrscheinlichkeit,
        # die unser RNN dem tatsächlichen nächsten Zeichen zuschreibt.
        # D.h. diese Zielfunktion führt dazu, dass die Vorhersage unseres Netzwerkes die tatsächliche
        # Wahrscheinlichkeitsverteilung für das nächste Zeichen, gegeben eine bestimmte Sequenz
        # unserer Trainingsdaten, immer besser approximiert.
        loss = loss_fn(torch.squeeze(output.reshape(-1,num_chars)), torch.squeeze(target_seq.reshape(-1)))
        running_loss += loss.item()

        # Sobald wir die Zielfunktion berechnet haben, macht PyTorch den Rest für uns.
        # Dafür resetten wir zunächst den Optimizer
        optimizer.zero_grad()
        
        # Dann berechnen wir die Gradienten der Zielfunktion für die einzelnen Netzwerkgewichte.
        loss.backward()
        
        # Und dann verändern wir die einzelnen Netzwerkgewichte mit Hilfe der Gradienteninformation 
        # ein bisschen.
        optimizer.step()
        
        # Damit haben wir einen weiteren Trainingsschritt durchgeführt und zählen unseren
        # Zähler eins hoch.
        steps_done += 1

        # Jetzt bewegen wir alle Leseköpfe um die Länge der gerade gelesenen Sequenz weiter
        data_ptr += seq_len
        
        # Wir müssen aufpassen: Sobald einer der Leseköpfe über das Ende der Trainingsdaten 
        # hinauslesen würde, setzen wir diesen Lesekopf an den Anfang der Trainingsdaten zurück
        # und löschen das Gedächtnis des RNNs für diesen Lesekopf.
        
        # Dazu gehen wir alle Leseköpfe durch
        for j in range(num_heads):
            
            # Wenn das Ende der nächsten gelesenen Sequenz hinter dem
            # Ende der Trainingsdaten liegt.
            if data_ptr[j] + seq_len + 1 > train_data_size:

                # Dann setzen wir den Lesekopf wieder an eine Position
                # am Anfang des Textes (die genau der Anzahl der 
                # "über das Ziel hinaus geschossenen" Zeichen
                # entspricht)
                data_ptr[j] = ( data_ptr[j] + seq_len ) % train_data_size

                # Für diesen Lesekopf löschen wir natürlich das entsprechende
                # Gedächtnis, in dem wir die zugehörigen Einträge im hidden_state
                # wieder auf den Startwert 0.0 setzen.
                hidden_state[0][:,j,:] = 0.0
                hidden_state[1][:,j,:] = 0.0
                
        # Alle sample_every Schritte generieren wir etwas Text, damit wir
        # zusehen können, wie unser RNN die Struktur der Texte lernt.
        if steps_done % sample_every == 0:
            print('Sampling some text:')
            sample(rnn, device, prompt = sampling_prompt)

            
        # Alle log_every Schritte speichern wir die aktuelle Zielfunktion in 
        # einer Textdatei
        if steps_done % log_every == 0:
            validation_loss = calculate_loss_on_dataset(valid_data, rnn, num_heads, seq_len, device)
            log_metrics_to_file('log' + model_name + '.txt', [float(steps_done) / iter_per_epoch, loss.item(), validation_loss], steps_done == log_every)
            torch.save(rnn.state_dict(), save_path)   

    # print loss and save weights after every epoch
    print("Epoch: {0} \t Loss: {1:.8f}".format(i_epoch, running_loss/iter_per_epoch))
    

# Wie geht es weiter?
Lassen sie dieses Notebook in einem Tab offen und öffnen sie das Notebook **02_Evaluate_RNN_Training**. In diesem Notebook können sie den Fortschritt des Trainings (in dem sie den entsprechenden Modellnamen angeben, den sie sich ausgesucht haben) beobachten. Dieses Notebook zeichnet ihnen auch den Kurvenverlauf der Zielfunktion auf den Trainings- und den Validierungsdaten, so dass sie das Training rechtzeitig beenden können, in dem sie in diesem Notebook hier in der Menüzeile oben "Kernel -> Interrupt" auswählen (oder auf das Schwarze Quadrat rechts neben dem Run-Button klicken). Im Notebook **02_Evaluate_RNN_Training** können sie außerdem die Zielfunktion auf dem Testset berechnen **und** mit ihrem trainierten Netzwerk Text generieren.

# Wie kann ich so ein Netzwerk einfach trainieren und in ein eigenes Projekt (z.B. einen Chatbot) integrieren?

Wir stellen in der Datei RNN_helper_functions.py alle Funktionen zur Verfügung, die sie brauchen, um einfach und schnell eigene RNNs zu Trainieren, das Training zu überwachen und mit den trainierten Netzwerken Texte zu erzeugen. Für einen Überblick, wie das geht, schauen sie bitte in das Notebook **03_Easy_to_use_RNNs**.